In [60]:
import sys
import csv
import requests
import re
from bs4 import BeautifulSoup

In [61]:
def load_page(url):
    with requests.get(url) as f:
        page = f.text
    return page

In [62]:
def get_element_text(element):
    try:
        return element.text.strip()
    except AttributeError as e:                     # This is the error python raises if a method is not available
        print('Element not found, error: {}'.format(e), file=sys.stderr)
        return ''

In [63]:
def get_song_info(url):
    song_page = BeautifulSoup(load_page('https://www.azlyrics.com/lyrics/charlieputh/marvingaye.html'), 'lxml')                  # Parse the requested page
    interesting_html = song_page.find(class_= 'row')                       # Get the part of the page where the interesting information is 'stored'
    if not interesting_html:                                           # Check if an article tag was found, not all pages have one
        print('No information availible for song at {}'.format(url), file=sys.stderr)
        return {}                                                      # Return an empty dictionary
    credits = get_element_text(interesting_html.find(class_='lyricsh')) # title = interesting_html.find('h2').text.strip() # We already have this information
    lyrics = song_page.find_all('div')[10].text.strip()
    lyrics = lyrics.replace('\r\n', '\n')                              # Windows newline to Unix newline
    # After each song, there are a bunch of tabs, and a copyright notice.
    # We want that notice on a new line, as it is cleaner. To do this, we
    # can use a regular expression which finds any occurence of two or more
    # tabs directly after each other: '\t\t+'. The first \t is the first tab,
    # the second \t is the second tab, and the + means: 1 or more times, and
    # applies to the previous character (the second tab).
    lyrics = re.sub('\t\t+', '\n', lyrics)
    return {'lyricsh': credits, 'div': lyrics}   

In [64]:
song_url = 'https://www.azlyrics.com/lyrics/charlieputh/marvingaye.html'
song_info = get_song_info(song_url)
for key, value in song_info.items():
    if key == 'div':
        print(value)

"Marvin Gaye" lyrics

Charlie Puth Lyrics




"Marvin Gaye"
(feat. Meghan Trainor)



Let's Marvin Gaye and get it on
You got the healing that I want
Just like they say it in the song
Until the dawn, let's Marvin Gaye and get it on

We got this king size to ourselves
Don't have to share with no one else
Don't keep your secrets to yourself
It's Kama Sutra show and tell (yeah)
Whoa
There's loving in your eyes
That pulls me closer
(Ooh, it pulls me closer)
It's so subtle
(It's so subtle)
I'm in trouble
(I'm in trouble)
But I'd love to be in trouble with you

Let's Marvin Gaye and get it on
You got the healing that I want
Just like they say it in the song
Until the dawn, let's Marvin Gaye and get it on
You got to give it up to me
I'm screaming mercy, mercy please
Just like they say it in the song
Until the dawn, let's Marvin Gaye and get it on

[Meghan Trainor:]
And when you leave me all alone
I'm like a stray without a home
I'm like a dog without a bone
I just want you for my own
I got to

Element not found, error: 'NoneType' object has no attribute 'text'


In [78]:
def get_songs(url):
    index_page = BeautifulSoup(load_page(url), 'lxml')        # Parse the page
    items = index_page.find(id='listAlbum')                   # Get the list from the webpage
    if not items:                                             # If the webpage does not contain the list, we should exit
        print('Something went wrong!', file=sys.stderr)
        sys.exit()
    data = []
    for row in items.find_all(class_='listalbum-item'):          # Go over the entries line-by-line
        # Get the text in the first element with the class 'song'
        # Note: we use class_ with an underscore, as class is already a
        # reserved python keyword. This means we can't use it as a
        # variable name.
        song = row.find(class_='listalbum-item')[1].text.strip() # Link text
        link = row.find(class_='listalbum-item')[0].get('href') 
        #I am not sure if these two steps are correct
        
        
        
        #album_name = row.find(class_='release').text.strip()
        #first_played = row.find_all(class_='played')[0].text.strip()
        #last_played = row.find_all(class_='played')[1].text.strip()
       # times_played = row.find(class_='times').text.strip()
        data.append({                                         # Store the data in a dictionary, and add that to our list
                     'song': song,
                     'link': link,
                    # 'album name': album_name,
                    # 'first played': first_played,
                    # 'last played': last_played,
                    # 'times played': times_played
                    })
    return data

In [79]:
index_url = 'https://www.azlyrics.com/c/charlieputh.html'         # Contains a list of songs
song_data = get_songs(index_url)                      # Get songs with metadata
for row in song_data:
    print('Scraping info on {}.'.format(row['listalbum-item']) # Might be useful for debugging
    url = row['link']
    song_info = get_song_info(url)                    # Get lyrics and credits for this song, if available
    for key, value in song_info.items():
        row[key] = value        

SyntaxError: invalid syntax (1132005124.py, line 5)